<a href="https://colab.research.google.com/github/akshaybroota/learning-backpropagation/blob/main/re_implementing_micro_grad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code


In [48]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
%matplotlib inline

In [45]:
#Creating karpathy@'s micro-grad engine using tanh vs reLU

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._prev = set(_children)
    self._backward = lambda : None
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1 * out.grad
      other.grad += 1* out.grad
    out._backward = _backward

    return out

  def __neg__(self):
    return self * -1

  def __radd__(self, other):
    return self + other

  def __sub__(self, other):
    return self + (-other)

  def __rsub__(self, other):
    return other + (-self)

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __rtruediv__(self, other): # other / self
    return other * self**-1

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __rmul__(self, other):
    return self * other

  def __pow__(self, other):

    assert isinstance(other, (int, float))
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
      self.grad += (other * self.data**(other-1)) * out.grad
    out._backward = _backward

    return out

  def relu(self):
    out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

    def _backward():
      self.grad = (out.data > 0) * out.grad
    out._backward = _backward

    return out

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad = (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()


In [49]:
#creating PyTorch like Neural Net library based on Karpathy@'s micro-grad lib

class Neuron:
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1, 1))

  def __call__(self, x):
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out

  def parameters(self):
    return self.w + [self.b]

class Layer:
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [55]:
# Load the Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Convert to a binary classification problem (Iris-setosa vs. not Iris-setosa)
X = X[y != 2]
y = y[y != 2]
y[y == 0] = 0
y[y == 1] = 1

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Normalization
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std



In [56]:
#defining the loss function and calculate accuracy

def loss(y_pred, y_true):
  #setting threshold for binary classification
  threshold = 0.5
  for pred in y_pred:
    if pred.data > threshold:
      pred.data = 1
    else:
      pred.data = 0

  #Calculating MSE
  losses = [((yt - yp)**2) for yt, yp in zip(y_true, y_pred)]
  data_loss = sum(losses) * (1.0 / len(losses))
  accuracy = [(yi > threshold) == (scorei.data > threshold) for yi, scorei in zip(y_true, y_pred)]
  acc = np.mean(accuracy)
  return data_loss, acc


In [57]:
#Create an MLP model

model = MLP(4, [16, 16, 1])

In [58]:
# training the model
for k in range(25):

    # forward
    predictions = list(map(model, X_train))
    actual_loss, acc = loss(predictions, y_train)

    # backward
    for p in model.parameters():
        p.grad = 0.0
    actual_loss.backward()

    # update (sgd)
    learning_rate = 0.05
    for p in model.parameters():
      p.data += -learning_rate * p.grad


    print(f"step {k} loss {actual_loss.data}, accuracy {acc*100}%")

running backward
step 0 loss 0.7625000000000001, accuracy 23.75%
running backward
step 1 loss 0.65, accuracy 35.0%
running backward
step 2 loss 0.5625, accuracy 43.75%
running backward
step 3 loss 0.5750000000000001, accuracy 42.5%
running backward
step 4 loss 0.525, accuracy 47.5%
running backward
step 5 loss 0.4625, accuracy 53.75%
running backward
step 6 loss 0.30000000000000004, accuracy 70.0%
running backward
step 7 loss 0.21250000000000002, accuracy 78.75%
running backward
step 8 loss 0.15000000000000002, accuracy 85.0%
running backward
step 9 loss 0.125, accuracy 87.5%
running backward
step 10 loss 0.1, accuracy 90.0%
running backward
step 11 loss 0.037500000000000006, accuracy 96.25%
running backward
step 12 loss 0.0125, accuracy 98.75%
running backward
step 13 loss 0.0125, accuracy 98.75%
running backward
step 14 loss 0.0125, accuracy 98.75%
running backward
step 15 loss 0.0125, accuracy 98.75%
running backward
step 16 loss 0.0125, accuracy 98.75%
running backward
step 17 loss

In [59]:
#using test_data
test_predictions = list(map(model, X_test))
test_loss, test_acc = loss(test_predictions, y_test)
print(f"loss {test_loss.data}, accuracy {test_acc*100}%")

loss 0.0, accuracy 100.0%
